### 📘 Video 3: Query a PDF (Ask Questions About Any Document)

**Goal**: Upload a PDF and ask questions — teach your AI to read and understand documents.

✅ No API keys

✅ Uses Hugging Face + LangChain

✅ Runs on free Colab GPU

📄 Let's make AI work for *your* data!

In [1]:
# Install required libraries
!pip install -q langchain-huggingface langchain-community transformers torch accelerate bitsandbytes pdfplumber

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 29.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 69.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 65.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


### 📚 Why Query a PDF?

You have documents:
- Research papers
- Contracts
- Manuals
- Articles

But who has time to read them?

👉 Teach your AI to **read, remember, and answer** — privately and for free.

💡 This is **RAG (Retrieval-Augmented Generation)** — a core pattern in Doug Ortiz’s AI & LLM work.

In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain_huggingface import HuggingFacePipeline

# Load lightweight model
model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    torch_dtype="auto",
    trust_remote_code=True
)

# Create text generation pipeline
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=100,
    temperature=0.7,
    do_sample=True
)

# Wrap with LangChain
llm = HuggingFacePipeline(pipeline=pipe)

print("✅ Model loaded and ready!")

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Device set to use cpu


✅ Model loaded and ready!


### 📎 Upload a PDF

Click below to upload any PDF (e.g., a research paper, manual, or article).

In [3]:
from google.colab import files

uploaded = files.upload()
pdf_path = list(uploaded.keys())[0]
print(f"Uploaded: {pdf_path}")

Saving GoLang-AI Production Hero_dougortiz.pdf to GoLang-AI Production Hero_dougortiz.pdf
Uploaded: GoLang-AI Production Hero_dougortiz.pdf


### 🧩 Extract Text from PDF

In [4]:
import pdfplumber

# Extract text from PDF
def extract_text_from_pdf(pdf_path):
    text = ""
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            extracted = page.extract_text()
            if extracted:
                text += extracted + "\n"
    return text

raw_text = extract_text_from_pdf(pdf_path)

# Use first ~2000 chars to fit context and keep it fast
context = raw_text[:2000]

print(f"✅ Extracted {len(raw_text)} characters. Using first 2000 for speed.")
print(f"Preview: {context[:300]}...\n")

✅ Extracted 10265 characters. Using first 2000 for speed.
Preview: Go: AI Production Hero
Author: Doug Ortiz
linkedin.com/in/doug-ortiz-illustris 2025
dougortiz.blogspot.com
CONTENTS
01 Python Dominance
02 Go Emergence
03 Core Strengths
04 Language Duel
05 Production Playbook
06 Future Outlook
Author: Doug Ortiz
linkedin.com/in/doug-ortiz-illustris
dougortiz.blogsp...



### 💬 Ask Questions About the PDF

In [ ]:
def query_pdf(question, context):
    prompt = f"<|im_start|>system\nYou are a helpful assistant. Answer based only on the provided text. Be concise.\n<|im_end|>\n<|im_start|>user\nDocument excerpt:\n{context}\n\nQuestion: {question}<|im_end|>\n<|im_start|>assistant\n"
    response = llm.invoke(prompt)
    # Clean response
    answer = response.replace(prompt, "").replace("<|im_start|>assistant\n", "").replace("<|im_end|>", "").strip()
    return answer

# Try a question
question = "What is this document about?"
answer = query_pdf(question, context)

print(f"Q: {question}")
print(f"A: {answer}\n")

# Another example
question2 = "Summarize the main points in 2 sentences."
answer2 = query_pdf(question2, context)

print(f"Q: {question2}")
print(f"A: {answer2}")

Q: What is this document about?
A: This document is called "AI Production Hero" and it is a comprehensive guide on how to build and deploy AI systems for production use. It covers topics such as Python dominance, Go emergence, core strengths, language duel, and production playbook. The guide is written by a seasoned AI engineer, Doug Ortiz, and offers valuable insights and strategies for implementing AI in production environments.



### 🎉 Summary

In this notebook, you:

✅ Uploaded a PDF

✅ Extracted text using `pdfplumber`

✅ Asked questions using your local LLM

💡 This is **RAG (Retrieval-Augmented Generation)** — a foundational AI pattern used in real-world systems

➡️ **Next: Give your AI web search — so it knows current events!**

### 🔗 Resources

- [LangChain Docs](https://python.langchain.com)
- [TinyLlama on Hugging Face](https://huggingface.co/TinyLlama/TinyLlama-1.1B-Chat-v1.0)
- [pdfplumber on PyPI](https://pypi.org/project/pdfplumber/)
- [Author: Doug Ortiz](https://www.linkedin.com/in/doug-ortiz-illustris/)
- [YouTube Channel: @techbits-do](https://www.youtube.com/@techbits-do)
- [Illustris.org](https://www.illustris.org)